In [1]:
import pandas as pd
import glob
from statsmodels.tsa.api import SimpleExpSmoothing

In [2]:
cols = ['Year', 'Reporter Code', 'Reporter', 'Partner Code', 'Partner', 'Trade Value (US$)']
schema = {'Year': str, 'Reporter Code': str, 'Partner Code': str}
def dfs():
    for file in glob.glob('data/*.csv'):
        yield pd.read_csv(file, encoding='latin-1', usecols=cols, 
                          dtype=schema)
df = pd.concat(dfs())
df.dropna(inplace=True)
df.rename(columns={'Reporter Code': 'Reporter_Code', 'Partner Code': 'Partner_Code', 'Trade Value (US$)': 'Trade_Value'}, inplace=True)

In [3]:
reporters = set(df.Reporter_Code.unique())
print(len(reporters), "Source nodes are present. (Some have invalid data so dropped)")
df_ = df[df.Partner_Code.isin(reporters)]  #induced sub-graph creation
print("Filtered records from", len(df), "to", len(df_))

143 Source nodes are present. (Some have invalid data so dropped)
Filtered records from 74077 to 66316


In [4]:
# Filtered DF is essentially an induced graph now. Let's save this
df_.to_csv('filtered_df.csv', header=True)

In [5]:
countries = df_.Reporter.unique()
country_to_index = dict(zip(countries, range(len(countries))))
index_to_country = dict(map(reversed, country_to_index.items()));

In [7]:
# We can now start creating tensors
import numpy as np
# First dimension is for time (10 years)
# Second dimension is for TO
N = len(countries)
tensor = np.zeros((10, N, N))

In [8]:
for i, year in enumerate(['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']):
    # Data of that particular year
    df__ = df_[df_.Year == year]
    # For the scale of this data, I think we can just iterate over it.
    for _, row in df__.iterrows():
        country_from = row['Partner']
        country_to = row['Reporter']
        tensor[i, country_to_index[country_from], country_to_index[country_to]] = row['Trade_Value']

In [9]:
sparsity = 1.0 - np.count_nonzero(tensor) / tensor.size
print(f"{sparsity*100:.2f}% of the tensor is sparse")

67.58% of the tensor is sparse


In [176]:
tensor

array([[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 2.2651200e+05, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 3.5345950e+06],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 1.6782854e+07, 0.0000000e+00]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 1.5125220e+06, 0.0000000e+00, ...,
         0.000

In [168]:
!pip install tensorly

In [11]:
# Normalizing our tensor. For the first version, let's just calculate the log
non_zeros = (tensor!=0)
tensor[non_zeros] = np.log(tensor[non_zeros])
tensor

array([[[16.53887063,  0.        , 12.35463919, ...,  0.        ,
          4.76217393,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [11.60730794,  0.        ,  0.        , ...,  0.        ,
          4.83628191,  0.        ],
        [20.9365733 ,  0.        ,  0.        , ..., 18.05097911,
          0.        ,  0.        ],
        [13.00692559,  0.        , 11.15990099, ..., 13.50037369,
         14.15336398,  0.        ]],

       [[12.07779599,  0.        ,  9.23386157, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 8.36706773,  0.        ,  0.        , ...,  

In [12]:
from tensorly.decomposition import parafac

# Can tinker with ranks
# Mask has a special significance, we should really discuss and explore more about it
weights, factors = parafac(tensor, rank=150, normalize_factors=True, mask=non_zeros, random_state=42)

def get_slice(A, B, T, W):
    # Given the two matrices, get the slice
    print("Shape of A", A.shape)
    print("Shape of B", B.shape)
    print("Shape of T", T.shape)
    print("Shape of W", W.shape)
    print ("Shape of np.diag(W*T)", np.diag(W*T).shape)
    return np.matmul(np.matmul(A, np.diag(W*T)), B.T)

sl = get_slice(factors[1], factors[2], factors[0][0], weights)
sl[non_zeros[0]]

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)


array([14.34832391, 11.78516321,  5.81129014, ..., 11.64427857,
       13.20859964, 14.97849681])

In [34]:
sl.shape[0]

143

In [52]:
# based on paper
# factor matrix C
# predict same col in next time slice C(K+1)
# takes in factors produced by parafac and years to forecast (default is 1) 
# returns list of forecasted vectors, last n in list is the forecasted n years

def exponential_smoothing(factors, weights, forecast = 1):
    num_slices = len(factors[0])
    alpha = 0.5
    
    sl = get_slice(factors[1], factors[2], factors[0][0], weights)
    forecasts = [sl]
    
    for i in range(0, num_slices + forecast):
        # get slice
        s_t = forecasts[i]
        s_t1 = np.zeros((s_t.shape[0], s_t.shape[1]))
        
        past_c_k_r = []
        for k in range(0, s_t.shape[0]):
            
            for r in range(0, s_t.shape[1]):
                c_k_r = s_t[k][r]
                next_c_k_r = 0
                           
                for num in range(0, k):
                    smooth = alpha * ((1-alpha) ** num)
                    next_c_k_r += smooth * past_c_k_r[-num]
                
                past_c_k_r.append(next_c_k_r)
                    
                s_t1[k,r] = next_c_k_r
                
        forecasts.append(s_t1)
        
    return forecasts
         
        
        

In [53]:
# testing expo smoothing
forecasts = exponential_smoothing(factors, weights)
print(forecasts[-1])

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [54]:
forecasts[2]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

array([ 6.08449941, 17.9035443 , 12.39185072, ..., 16.69321177,
        9.42843118, 16.63586833])